In [1]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from sqlalchemy import inspect

import pandas as pd

import matplotlib.pyplot as plt



In [2]:
engine = create_engine("sqlite:///archive/cfb.db")
Base = automap_base()
Base.prepare(engine,reflect=True)
tables = Base.classes.keys()

session = Session(engine)

In [26]:
cfb = pd.DataFrame()

for table in tables:
    temp = pd.DataFrame(session.execute(f'select * from {table}'))
    keys = session.execute(f'select * from {table}').keys()
    temp.columns = keys
    temp['Year'] = table.removeprefix('cfb')
    cfb = pd.concat([cfb,temp])

cfb.head()

,Team,Games,Win,Loss,Off.Rank,Off.Plays,Off.Yards,Off.Yards.Play,Off.TDs,Total.TDs,...,Turnover.Margin,Avg.Turnover.Margin.per.Game,Year,...40,Kickoff.Return.Def.Rank,Opp.Kickoff.Returns,Kickoff.Touchbacks,Opponent.Kickoff.Return.Yards,Opp.Kickoff.Return.Touchdowns.Allowed,Avg.Yards.per.Kickoff.Return.Allowed
0,Akron (MAC),12,5,7,106,837,4104,4.9,30,31,...,-2,-0.17,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alabama (SEC),13,11,2,33,826,5903,7.15,58,65,...,2,0.15,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arizona (Pac-12),13,8,5,31,1030,5960,5.79,52,57,...,4,0.31,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arizona St. (Pac-12),14,10,4,32,1102,6402,5.81,64,69,...,15,1.07,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arkansas (SEC),12,3,9,99,775,4286,5.53,29,30,...,-9,-0.75,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
cfb[['Team','Conference','NA']] = cfb['Team'].str.split('(',expand=True)
cfb['Team'] = cfb['Team'].str.replace(' ','')
cfb['Conference'] = cfb['Conference'].str.replace(' ','')
cfb['Conference'] = cfb['Conference'].str.replace(')','')

cfb.loc[cfb['Conference'] =='OH','Team'] = 'Miami (OH)'
cfb.loc[cfb['Conference'] =='FL','Team'] = 'Miami (FL)'
cfb.loc[cfb['Conference'] =='OH','Conference'] = 'MAC'
cfb.loc[cfb['Conference'] =='FL','Conference'] = 'ACC'

cfb = cfb.dropna(1,'any')

cfb['Conference'] = cfb['Conference'].str.replace('MountainWest','MWC')
cfb['Conference'] = cfb['Conference'].str.replace('FBSIndependent','Independent')

cfb.loc[(cfb['Team'] == 'OleMiss'),'Conference'] = 'SEC'
cfb.loc[(cfb['Team'] == 'Pittsburgh'),'Conference'] = 'ACC'

cfb.head()

C:\Users\ztjoh\AppData\Local\Temp/ipykernel_16912/1772927818.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  cfb['Conference'] = cfb['Conference'].str.replace(')','')
C:\Users\ztjoh\AppData\Local\Temp/ipykernel_16912/1772927818.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  cfb = cfb.dropna(1,'any')


,Team,Games,Win,Loss,Off.Rank,Off.Plays,Off.Yards,Off.Yards.Play,Off.TDs,Off.Yards.per.Game,...,Fumbles.Recovered,Opponents.Intercepted,Turnovers.Gain,Fumbles.Lost,Interceptions.Thrown.y,Turnovers.Lost,Turnover.Margin,Avg.Turnover.Margin.per.Game,Year,Conference
0,Akron,12,5,7,106,837,4104,4.9,30,342,...,6,13,19,11,10,21,-2,-0.17,2013,MAC
1,Alabama,13,11,2,33,826,5903,7.15,58,454.1,...,8,11,19,10,7,17,2,0.15,2013,SEC
2,Arizona,13,8,5,31,1030,5960,5.79,52,458.5,...,4,18,22,11,7,18,4,0.31,2013,Pac-12
3,ArizonaSt.,14,10,4,32,1102,6402,5.81,64,457.3,...,12,21,33,6,12,18,15,1.07,2013,Pac-12
4,Arkansas,12,3,9,99,775,4286,5.53,29,357.2,...,6,8,14,12,11,23,-9,-0.75,2013,SEC


In [28]:
to_num = cfb.columns.drop(['Team','Conference','Average.Time.of.Possession.per.Game','Time.of.Possession'])

cfb[to_num] = cfb[to_num].apply(pd.to_numeric)
cfb.dtypes

Team                             object
Games                             int64
Win                               int64
Loss                              int64
Off.Rank                          int64
                                 ...   
Turnovers.Lost                    int64
Turnover.Margin                   int64
Avg.Turnover.Margin.per.Game    float64
Year                              int64
Conference                       object
Length: 147, dtype: object

In [29]:
cfb[cfb['Conference']=='']

,Team,Games,Win,Loss,Off.Rank,Off.Plays,Off.Yards,Off.Yards.Play,Off.TDs,Off.Yards.per.Game,...,Fumbles.Recovered,Opponents.Intercepted,Turnovers.Gain,Fumbles.Lost,Interceptions.Thrown.y,Turnovers.Lost,Turnover.Margin,Avg.Turnover.Margin.per.Game,Year,Conference


In [30]:
cfb.to_csv('cfb_combined.csv')